<a href="https://colab.research.google.com/gist/dsaint31x/2973d9478c5cd9bb188d66095539f02d/pillow_basic03_image_enhance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Pillow 사용법 - Basic 03 - Image Enhance](https://ds31x.tistory.com/466)



## Pillow - ImageEnhance 모듈

### Image Enhancement란?
영상에서 원하는 정보를 사람이 더 쉽게 해석할 수 있도록, **Brightness, Contrast, Spatial Characteristics** 등을 개선해 **주관적 화질을 향상**시키는 과정입니다.

### 사용방법
1. `ImageEnhance.XXX(img)`로 대상 이미지에 대한 Enhance 객체 생성
2. `enhance(factor)` 메서드를 호출하여 Image Enhancement 수행
   - `factor=1.0`: 원본 유지
   - `factor>1.0`: 강화
   - `factor<1.0`: 약화

### 지원되는 Enhance 종류
- **Brightness**: 밝기
- **Contrast**: 대비
- **Color**: 색의 채도(Saturation)
- **Sharpness**: 선명도

---


In [ ]:
from skimage import data
from PIL import Image, ImageEnhance

In [ ]:
img_array = data.astronaut()
img_array.shape

In [ ]:
def combine_img(img0, img1):
  w,h  = img0.size
  img1 = img1.resize((w,h))
  combined = Image.new("RGB", (w*2, h))
  combined.paste(img0, (0,0))
  combined.paste(img1, (w,0))
  return combined

In [ ]:
img = Image.fromarray(img_array)
img

### 1. Brightness (밝기)

**수식:** `I_out = I_in × factor`

**파라미터:**
- `factor=1.0`: 원본 brightness
- `factor<1.0`: 어둡게
- `factor>1.0`: 밝게

```python
enhancer_bright = ImageEnhance.Brightness(img)
img_bright = enhancer_bright.enhance(1.5)  # 1.5배 밝게
```


In [ ]:
enhancer_bright = ImageEnhance.Brightness(img)
img_bright = enhancer_bright.enhance(1.5)  # 1.5배 밝게
print(f"{type(enhancer_bright) = }")
combine_img(img,img_bright)

In [ ]:
enhancer_contrast = ImageEnhance.Contrast(img)
img_contrast = enhancer_contrast.enhance(0.8)  # 대비 줄이기
combine_img(img, img_contrast)

### 2. Contrast (대비)

**수식:** `I_out = (I_in - μ) × factor + μ`

여기서 μ는 대상 이미지의 평균 brightness입니다.

**파라미터:**
- `factor=1.0`: 원본 contrast
- `factor<1.0`: contrast 낮추기
- `factor>1.0`: contrast 증가

```python
enhancer_contrast = ImageEnhance.Contrast(img)
img_contrast = enhancer_contrast.enhance(0.8)  # 대비 줄이기
```


In [ ]:
enhancer_color = ImageEnhance.Color(img)
img_color = enhancer_color.enhance(2.0)  # saturation 두 배로 진하게
combine_img(img, img_color)

In [ ]:
enhancer_sharpness = ImageEnhance.Sharpness(img)
img_sharp = enhancer_sharpness.enhance(5.)  # sharpening
combine_img(img,img_sharp)

### 3. Color - Saturation (채도)

**수식:** `I_out = μ_gray + (I_in - μ_gray) × factor`

여기서 μ_gray는 픽셀의 그레이스케일 값입니다 (예: `0.299R + 0.587G + 0.114B`).

**파라미터:**
- `factor=0`: 흑백으로 변환
- `factor=1`: 원본 색감
- `factor>1`: 채도 증가

```python
enhancer_color = ImageEnhance.Color(img)
img_color = enhancer_color.enhance(2.0)  # saturation 두 배로 진하게
```


In [ ]:
# RGB → YCbCr 변환 후 채널 분리
img_ycbcr = img.convert("YCbCr")
y, cb, cr = img_ycbcr.split()

# 1) 밝기 조절
enhancer_b = ImageEnhance.Brightness(y)
y_bright = enhancer_b.enhance(2)  # 밝기 2배

# 수정한 Y와 원래 Cb, Cr로 YCbCr 이미지 복원
ycbcr_mod = Image.merge("YCbCr", (y_bright, cb, cr))

# YCbCr → RGB 변환
img_result = ycbcr_mod.convert("RGB")

combine_img(img,img_result)

In [ ]:
# RGB → YCbCr 변환 후 채널 분리
img_ycbcr = img.convert("YCbCr")
y, cb, cr = img_ycbcr.split()

# 2) 대비 조절
enhancer_c = ImageEnhance.Contrast(y)
y_contrast = enhancer_c.enhance(.1)  # 대비 1.5배

# 수정한 Y와 원래 Cb, Cr로 YCbCr 이미지 복원
ycbcr_mod = Image.merge("YCbCr", (y_contrast, cb, cr))

# YCbCr → RGB 변환
img_result = ycbcr_mod.convert("RGB")

combine_img(img,img_result)

### 4. Sharpness (선명도)

**수식:** `I_out = I_in × factor + I_blur × (1 - factor)`

여기서 I_blur는 blurred image입니다.

**파라미터:**
- `factor=0`: blurring
- `factor=1`: 원본
- `factor>1`: sharpening

```python
enhancer_sharpness = ImageEnhance.Sharpness(img)
img_sharp = enhancer_sharpness.enhance(5.)  # sharpening
```


### YCbCr 색공간에서 Enhancement

YCbCr 색공간에서는 밝기(Y 채널)와 색상(Cb, Cr 채널)이 분리되어 있어, 밝기만 조절하거나 대비만 조절하는 것이 더 효과적입니다.

**방법:**
1. RGB → YCbCr로 변환
2. Y 채널 분리
3. Y 채널에만 Enhancement 적용
4. Y, Cb, Cr 병합 후 RGB로 변환

**밝기 조절 예시:**


**대비 조절 예시:**
